# Treinamento de Modelos

Este notebook implementa o ponto 4 do trabalho: separar treino e teste, treinar três modelos variados, ajustar hiperparâmetros e registrar métricas.

## Estratégia

- Uso da divisão `train_test_split`
- Três modelos diferentes: Random Forest, Gradient Boosting e HistGradientBoosting
- Otimização via `RandomizedSearchCV` e validação cruzada
- Armazenamento dos resultados para comparações posteriores

### Preparação do dataset

Lê o CSV consolidado, garante que todas as colunas estejam disponíveis e fornece um ponto de partida narrativo.

In [ ]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

DATA_FILE = Path('data/clean/olist_ml_ready.csv')
df = pd.read_csv(DATA_FILE)
print(f'Dataset pronto com {len(df)} registros.')

In [ ]:
drop_cols = [
    'order_id', 'customer_id',
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
TARGET = 'delivery_time_days'
X = df.drop(columns=drop_cols + [TARGET], errors='ignore')
y = df[TARGET]

### Seleção de features e alvo

Remove identificadores temporais e separa a variável `delivery_time_days` para que o treino use apenas features preditivas.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
print('Treino:', X_train.shape, 'Teste:', X_test.shape)

### Divisão treino/teste

Usa `train_test_split` com seed fixa para reproduzir resultados enquanto deixa 25% da base para validação.

In [ ]:
models = {
    'RandomForest': (
        RandomForestRegressor(random_state=42),
        {
            'n_estimators': [50, 70],
            'max_depth': [10, 15],
            'min_samples_leaf': [1, 2]
        }
    ),
    'GradientBoosting': (
        GradientBoostingRegressor(random_state=42),
        {
            'n_estimators': [40, 60],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 4]
        }
    ),
    'HistGradientBoosting': (
        HistGradientBoostingRegressor(
            random_state=42,
            early_stopping=True,
            validation_fraction=0.1,
            n_iter_no_change=10
        ),
        {
            'max_iter': [80, 100],
            'learning_rate': [0.05, 0.08],
            'max_depth': [10, 12]
        }
    )
}
searches = {}
for name, (estimator, params) in models.items():
    print(f'Executando {name}...')
    search = RandomizedSearchCV(
        estimator,
        params,
        n_iter=1,
        cv=2,
        scoring='neg_root_mean_squared_error',
        random_state=42,
        n_jobs=1
    )
    search.fit(X_train, y_train)
    searches[name] = search
    print(f'  Melhor RMSE CV: {-search.best_score_:.3f}')

### Busca e ajuste de hiperparâmetros

Experimenta 3 modelos com `RandomizedSearchCV`, definindo um espaço enxuto e early stopping para manter o custo computacional em cheque.

In [ ]:
metrics = []
for name, search in searches.items():
    preds = search.best_estimator_.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    metrics.append({
        'modelo': name,
        'rmse': rmse,
        'mae': mae,
        'r2': r2_score(y_test, preds)
    })
metrics_df = pd.DataFrame(metrics).sort_values('rmse')
metrics_df

### Métricas registradas

Calcula RMSE/MAE/R² no conjunto de teste para comparar os melhores estimadores de cada modelo.

In [ ]:
output_dir = Path('src/models/trained')
output_dir.mkdir(parents=True, exist_ok=True)
for name, search in searches.items():
    joblib.dump(search.best_estimator_, output_dir / f'{name}.joblib')
metrics_df.to_json(Path('src/models/training_metrics.json'), orient='records', indent=2, force_ascii=False)
print('Modelos e métricas salvos em src/models/trained e src/models/training_metrics.json')

### Salvando artefatos

Persistência dos modelos com `joblib` e das métricas em `training_metrics.json` para uso pelos notebooks de análise.